In [11]:
import pandas as pd
from datetime import datetime, timezone,timedelta,date
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import seaborn as sns
from itertools import islice
import json
from web3 import Web3
import matplotlib.ticker as mtick
path = ""
path_output=""

In [2]:
matcha_address=['0xdef1c0ded9bec7f1a1670819833240f027b25eff',
               '0x61935cbdd02287b511119ddb11aeb42f1593b7ef']

In [3]:
#2022 matcha involved transaction data
df_matcha_allTxs=pd.read_csv(path+'matcha_txs.csv.gz',header=None, names
                         =['block','index','tx_hash','from','to','value','transfer_count','input_data','transfers'])

In [4]:
df_matcha_allTxs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2551247 entries, 0 to 2551246
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   block           int64 
 1   index           int64 
 2   tx_hash         object
 3   from            object
 4   to              object
 5   value           object
 6   transfer_count  int64 
 7   input_data      object
 8   transfers       object
dtypes: int64(3), object(6)
memory usage: 175.2+ MB


In [7]:
df_to_matcha_allTxs=df_matcha_allTxs[df_matcha_allTxs['to'].isin(matcha_address)]

In [13]:
# 2022 matcha transactions as victims for sandwich attacks
# data origin from EigenPhi
df_matcha_sandwich_txhash=pd.read_csv(path_output+'matcha_victim_txlist.csv')
df_matcha_sandwich=pd.read_csv(path_output+'matcha_victim_txs.csv')

In [14]:
df_matcha_sandwich_txhash.count()/df_to_matcha_allTxs.count()['tx_hash']

Unnamed: 0    0.019245
tx_hash       0.019245
dtype: float64

In [16]:
#datetime format
df_matcha_sandwich['datetime']=df_matcha_sandwich['blockTimestamp'].apply(lambda x: 
                                                                        datetime.fromtimestamp(x,tz=timezone.utc))
df_matcha_sandwich['datetime'] = pd.to_datetime(df_matcha_sandwich['datetime']).dt.tz_localize(None)
df_matcha_sandwich=df_matcha_sandwich.reset_index()

/var/folders/6b/m6620wh92lnbqjd1dz1tb0n40000gn/T/ipykernel_4574/3930031972.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matcha_sandwich['datetime']=df_matcha_sandwich['blockTimestamp'].apply(lambda x:
/var/folders/6b/m6620wh92lnbqjd1dz1tb0n40000gn/T/ipykernel_4574/3930031972.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_matcha_sandwich['datetime'] = pd.to_datetime(df_matcha_sandwich['datetime']).dt.tz_localize(None)


In [18]:
# calculat the volume that has been attacked

In [47]:
df_matcha_sandwich_price=pd.read_csv(path_output+'matcha_sandwich_withPrice.csv')

In [26]:
# filtered the transfer data from the victim transaction data
df_all_transfer=df_to_matcha_allTxs['transfers'].astype('string').fillna('').apply(lambda x: x.split('|')).explode().reset_index()

df_all_transfer[['token','transfer_from','transfer_to','amount']]=df_all_transfer['transfers'].str.split(';', expand=True)

df_all_transfer[['tx_hash','from','to']]=df_to_matcha_allTxs.loc[df_all_transfer['index']].reset_index()[['tx_hash','from','to']]

df_victim_transfer=df_all_transfer[df_all_transfer['tx_hash'].isin(df_matcha_sandwich_txhash['tx_hash'])]

In [28]:
df_matcha_sandwich_flat = df_matcha_sandwich['victimTxs'].apply(lambda x: x.rstrip().split(' ')).explode().reset_index()
df_matcha_sandwich_flat[['transactionHash','lps']]=df_matcha_sandwich.loc[df_matcha_sandwich_flat['index']].reset_index()[['transactionHash','lps']]

In [29]:
df_merge_1=df_matcha_sandwich_flat[['transactionHash','victimTxs','lps']].merge(
    df_victim_transfer[['tx_hash','transfer_from','transfer_to','token','amount']],left_on='victimTxs',
    right_on='tx_hash',how='left')


In [30]:
df_merge_1['lps']=df_merge_1['lps'].apply(lambda x: x.split(' '))

df_merge_1=df_merge_1.explode('lps')

In [31]:
df_victim_volume=df_merge_1[
    (df_merge_1['lps']==df_merge_1['transfer_from']) | (df_merge_1['lps']==df_merge_1['transfer_to'])]

In [32]:
df_token_decimals=pd.read_csv(path+'token_decimals.csv')

df_token_decimals=df_token_decimals[['address','decimals']].groupby('address').agg({'decimals':'max'}).reset_index()

In [34]:
#calculate the victim volume based on the token price and transfer amount
df_victim_volume=df_victim_volume.merge(
    df_token_decimals[['address','decimals']],left_on='token',right_on='address',how='left')

df_victim_volume=df_victim_volume.merge(df_matcha_sandwich_price,on='transactionHash',how='left')

df_victim_volume=df_victim_volume[df_victim_volume['token_x']==df_victim_volume['token_y']].reset_index()

df_victim_volume['amount']=df_victim_volume['amount'].astype(float)*df_victim_volume['decimals'].apply(lambda x: 10**(-x))

df_victim_volume['price']=df_victim_volume['price'].astype(float)

df_victim_volume['V_volume']=df_victim_volume['price']*df_victim_volume['amount']

df_victim_volume=df_victim_volume.dropna()

df_victim_volume_final=df_victim_volume[['transactionHash','victimTxs','V_volume']].groupby(
    ['transactionHash','victimTxs']).agg({'V_volume':'max'}).reset_index()

def select_value(group):
    if '0xdac17f958d2ee523a2206206994597c13d831ec7' in group['token_y'].values:
        return group.loc[group['token_y'] == '0xdac17f958d2ee523a2206206994597c13d831ec7','V_volume'].values[0]
    elif '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2' in group['token_y'].values:
        return group.loc[group['token_y'] == '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2']['V_volume'].values[0]
    elif '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48' in group['token_y'].values:
        return group.loc[group['token_y'] == '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48']['V_volume'].values[0]
    elif '0x6b175474e89094c44da98b954eedeac495271d0f' in group['token_y'].values:
        return group.loc[group['token_y'] == '0x6b175474e89094c44da98b954eedeac495271d0f']['V_volume'].values[0] 
    elif '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599' in group['token_y'].values:
        return group.loc[group['token_y'] == '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599']['V_volume'].values[0] 
    else:
        return group['V_volume'].max()

df_victim_volume_final=df_victim_volume.groupby(
    ['transactionHash','victimTxs']).apply(select_value).reset_index()

In [44]:
df_victim_volume_final.to_csv(path+'matcha_victim_volume.csv')